In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# detecting and cropping faces

In [3]:
import os
import glob

In [4]:
from gaze import init, Data

args = init(batch_size=8, no_cuda=True, verbose=1,
            dataset_faces_folder='_cache_dataset_faces',
            dataset_folder='../dataset')
d = Data(args)

Found 4501 sample images;  3601  to train 900 to test


In [5]:
args

Namespace(N_cv=20, activation='relu', batch_size=8, conv1_bn_momentum=0.5, conv1_dim=9, conv1_kernel_size=8, conv2_bn_momentum=0.5, conv2_dim=36, conv2_kernel_size=12, crop=75, dataset_faces_folder='_cache_dataset_faces', dataset_folder='../dataset', dataset_test='dataset_test', dataset_train='dataset_train', dense_bn_momentum=0.5, dimension=30, do_adam=False, epochs=40, fullsize=75, log_interval=0, lr=0.01, mean=0.4, momentum=0.05, no_cuda=True, num_processes=1, seed=42, size=40, size_test_set=0.2, std=0.3, stride1=2, stride2=4, test_batch_size=16, verbose=1)

In [6]:
n_show = 10
for target in d.classes:
    for filename in glob.glob(os.path.join(args.dataset_folder, target) + '/*.png')[-n_show:]:
        print(filename)

In [7]:
import imageio
import time
from dev.LeCheapEyeTracker_copy.EyeTrackerServer import FaceExtractor
#from gaze import FaceExtractor #/!\ ImportError
FE = FaceExtractor()
timings = []
for target in d.classes:
    for filename in glob.glob(os.path.join(args.dataset_folder, target, '*.png')):
        filename_face = filename.replace(args.dataset_folder, args.dataset_faces_folder)
        if not os.path.exists(filename_face):
            print('Extracting face in ', filename_face)
            frame = imageio.imread(filename)
            t0 = time.time()
            #t, b, l, r = FE.get_bbox(frame)
            face = FE.face_extractor(frame)#, bbox=(t, b, l, r))
            
            if np.all(face) != None :

                t1 = time.time()
                timings.append(t1-t0)
                if False:
                    # Create figure and axes
                    fig, ax = plt.subplots(figsize=(15, 8))

                    # Display the image
                    ax.imshow(frame)

                    # Create a Rectangle patch
                    from matplotlib import patches
                    rect = patches.Rectangle((t, l), r-l, t-b, linewidth=1, edgecolor='r', facecolor='none')

                    # Add the patch to the Axes
                    ax.add_patch(rect)
                    plt.show()

                if True:
                    # Create figure and axes
                    fig, ax = plt.subplots(figsize=(15, 8))

                    # Display the cropped image
                    ax.imshow(face)

                    plt.show()                
                print('face.shape', face.shape)
                imageio.imwrite(filename_face, face) 


In [8]:
if len(timings)>0: 
    timings_ms = np.array(timings) * 1000
    print('timings in ms =', timings_ms.mean(), '+/-', timings_ms.std()) 

In [9]:
import os
fname = 'figures/dlib_timings'
if not os.path.exists(fname + '.pdf'):
    if len(timings)>0:
        fig, ax = plt.subplots(figsize=((8, 5)))
        n, bins, patches = ax.hist(timings_ms, bins=np.linspace(100, 200, 100), alpha=.4)
        ax.vlines(np.median(timings_ms), 0, n.max(), 'g', linestyles='dashed', label='median = %.3f ms' % np.median(timings))
        ax.set_xlabel('Timings (ms)')
        ax.set_ylabel('Smarts')
        ax.legend(loc='best')
        for ext in ['.pdf', '.png']:
            plt.savefig(fname + ext)        